In [1]:
!pip install --quiet transformers sacremoses


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import nltk
import re

from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
from sklearn.model_selection import StratifiedKFold
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk import tokenize

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
train_df = pd.read_csv('../datasets/train.csv')
test_df = pd.read_csv('../datasets/test.csv')

In [5]:
X_train, y_train = train_df.iloc[:10]['Content'].values.tolist(), train_df.iloc[:10]['Suspicious_Level']
X_test = test_df.iloc[:10]['Content'].values.tolist()

### Processing

In [6]:
# import spacy

# nlp_ru = spacy.load('ru_core_news_md', disable=["parser", "ner"])

# def text_processing(texts):
#     processed_texts = []
#     for text in texts:
#         doc = nlp_ru(text)
#         processed_text = " ".join([token.lemma_ for token in doc if not token.is_stop])
#         processed_texts.append(processed_text)
#     return processed_texts

In [7]:
# Download the Snowball stemmer for Russian language
nltk.download('stopwords')
nltk.download('punkt')

# Create a Snowball stemmer for Russian
stemmer = SnowballStemmer("russian")

def collapse_dots(input):
    # Collapse sequential dots
    input = re.sub("\.+", ".", input)
    # Collapse dots separated by whitespaces
    all_collapsed = False
    while not all_collapsed:
        output = re.sub(r"\.(( )*)\.", ".", input)
        all_collapsed = input == output
        input = output
    return output

def process_text(input):
    if isinstance(input, str):
        input = " ".join(tokenize.sent_tokenize(input))
        input = re.sub(r"http\S+", "", input)
        input = re.sub(r"\n+", ". ", input)
        for symb in ["!", ",", ":", ";", "?"]:
            input = re.sub(rf"\{symb}\.", symb, input)
        input = re.sub("[^а-яА-Яa-zA-Z0-9!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~ё]+", " ", input)
        input = re.sub(r"#\S+", "", input)
        input = collapse_dots(input)
        input = input.strip()
        # input = input.lower()
    return input

train_df["Content_processed"] = train_df["Content"].apply(process_text)
test_df["Content_processed"] = test_df["Content"].apply(process_text)

# Tokenize the text using NLTK for Russian language
train_df['Content_tokenized'] = train_df['Content_processed'].apply(lambda x: ' '.join([stemmer.stem(word) for word in word_tokenize(x, language='russian')]))
test_df['Content_tokenized'] = test_df['Content_processed'].apply(lambda x: ' '.join([stemmer.stem(word) for word in word_tokenize(x, language='russian')]))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danorel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/danorel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Translator & Fake detector

In [8]:
mname = "Helsinki-NLP/opus-mt-ru-en"
translation_tokenizer = AutoTokenizer.from_pretrained(mname)
translation_model = AutoModelForSeq2SeqLM.from_pretrained(mname)

# mname = "facebook/wmt19-en-ru"
# translation_tokenizer = FSMTTokenizer.from_pretrained(mname)
# translation_model = FSMTForConditionalGeneration.from_pretrained(mname)

### Fake detection

In [9]:
fake_detection_tokenizer = AutoTokenizer.from_pretrained("vikram71198/distilroberta-base-finetuned-fake-news-detection")
fake_detection_model = AutoModelForSequenceClassification.from_pretrained("vikram71198/distilroberta-base-finetuned-fake-news-detection")

In [10]:
def translate_and_predict_fake(X_train):
    y_pred = np.array([])
    
    for x_train in tqdm(X_train):
        x_train_translated_ids = translation_tokenizer(
            x_train,
            max_length=512,
            truncation=True,
            return_tensors="pt"
        ).input_ids

        x_train_output = translation_model.generate(input_ids=x_train_translated_ids)

        x_train_translated = translation_tokenizer.batch_decode(
            x_train_output, 
            skip_special_tokens=True
        )

        x_train_fake_detection_input = fake_detection_tokenizer(
            x_train_translated, 
            truncation=True,
            max_length=512,
            return_tensors='pt'
        )

        x_train_fake_detection_output = fake_detection_model(**x_train_fake_detection_input)["logits"]
        x_train_fake_detection_detached_output = x_train_fake_detection_output.detach()

        fake_detection_softmax = nn.Softmax(dim = 1)
        x_train_fake_detection_prediction_probabilities = list(fake_detection_softmax(x_train_fake_detection_detached_output).detach().numpy())

        x, y = x_train_fake_detection_prediction_probabilities[0]
        print(x_train_fake_detection_prediction_probabilities)
        
        y_sample = 1 if x < y else 3
        y_pred = np.append(y_pred, y_sample)
        
        print(f"Prediction for sentence {x_train_translated} is {y_sample}")
        
    return y_pred

### Cross-validation

In [11]:
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

In [12]:
X = train_df.iloc[50:100]['Content_processed']
y = train_df.iloc[50:100]['Suspicious_Level']

In [13]:
fold_number = 1
f1_score_folds = []

for train_index, val_index in skf.split(X, y):
    X_train, X_val = X.iloc[train_index].values.tolist(), X.iloc[val_index].values.tolist()
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    y_pred = translate_and_predict_fake(X_train)
    
    f1_score = sklearn.metrics.f1_score(y_train, y_pred, average='macro')
    f1_score_folds.append(f1_score)
    
    print(f"Fold {fold_number} - F1 Score: {f1_score}")
    print(sklearn.metrics.classification_report(y_pred, y_val))

    fold_number += 1
    
f1_mean_score = sum(f1_score_folds) / num_folds
print(f"F1 Score: {f1_mean_score}")

/Users/danorel/.pyenv/versions/disinformation_3.10/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
  2%|██▋                                                                                                        | 1/40 [00:01<00:41,  1.05s/it]

Prediction for sentence ['Vladimir Rogov reports that in the Rostov region, air defences have successfully struck two air targets.'] is 1


  5%|█████▎                                                                                                     | 2/40 [00:02<00:47,  1.24s/it]

Prediction for sentence ['Kreminna fights: the big ones destroy the enemy with C-60 anti-aircrafts. Fighters travel several times a day to perform tasks and destroy targets that are known by drone operators / TASS/.'] is 1


  8%|████████                                                                                                   | 3/40 [00:03<00:42,  1.14s/it]

...

Prediction for sentence ["An attempt to strike the bridge could be linked to pressure on the Russian leadership to extend the grain deal, which ends on 17 July. But this option is unlikely, because Ukrainian units have been instructed not to work in this direction for the time being. In this situation, the way in which it is presented in the Ukrainian media, especially in the context of the return of the leaders of Azov to Ukraine: supposedly a new reality for Crimea and Russia as a whole. This attack, like in the Rostov region, is more like a part of a strategic information operation whose purpose is to raise the moral and psychological state of the members of the US before a new phase of the offensive. It is no secret that the loss of the U.S.U.'s links involved in the front-line fighting is enormous. Some of the ones on the Zaporizsk track have already been sent to pre-assed. These strikes should convince both Ukrainian units and mercenaries that they are winning. This is a plan to 

 70%|██████████████████████████████████████████████████████████████████████████▏                               | 28/40 [02:49<01:21,  6.83s/it]

Prediction for sentence ["It doesn't take much for us to survive and win our common cause. Our common cause doesn't ask for huge sums, we don't need anyone to work hard and make serious contributions. Great things always start small. Let your donation be at least 10 or 50 rubles. But if a few concerned people come together, it'll already be a tangible support. SBERBANK map: 4276160925483621. Don't get past it. A couple dozen rubles won't even buy bread, it won't make you poor, but it'll give you a lot of support for all of us."] is 1


 70%|██████████████████████████████████████████████████████████████████████████▏                               | 28/40 [02:52<01:14,  6.17s/it]


KeyboardInterrupt: 